### need to convert the XLS/X files to CSV;
- checked with Pandas but get much more time, need to check another methods; 

In [26]:
# check if the libraries are available

def check_library_availability( required_libraries = []):
    results = {}
    
    for lib in required_libraries:
        try:
            __import__(lib)
            results[lib] = True
        except ImportError:
            results[lib] = False

    max_length = max(len(lib) for lib in required_libraries)
    for lib, available in results.items():
        status = "INSTALLED ✅" if available else "MISSING ❌"
        print(f"{lib.ljust(max_length)} : {status}")
        
check_library_availability(required_libraries = ['cgi', 'sys', 'io', 'csv', 'os', 'python_calamine'])

cgi             : INSTALLED ✅
sys             : INSTALLED ✅
io              : INSTALLED ✅
csv             : INSTALLED ✅
os              : INSTALLED ✅
python_calamine : INSTALLED ✅


### main XLS convertion logic

In [34]:
import python_calamine

def convert_excel_to_csv(input_excel_path: str, output_csv_path: str) -> None:
    def is_meaningful_row(row):
        return any(cell is not None and str(cell).strip() != "" for cell in row)
        
    with open(input_excel_path, 'rb') as excel_file:
        workbook = python_calamine.CalamineWorkbook.from_filelike(excel_file)
        sheet = workbook.get_sheet_by_index(0)
        data = sheet.to_python()
        
        if not data:
            return
        
        # Convert header row and data rows to strings where appropriate.
        headers = [str(cell) for cell in data[0]]
        # data_rows = data[1:]
        # data_rows = [row for row in data[1:] if any(str(cell).strip() for cell in row)]
        data_rows = [row for row in data[1:] if is_meaningful_row(row)]
        
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as f:
        # Build the header: first cell is left as is; subsequent cells wrapped in a single pair of double quotes.
        header_line = ";".join([headers[0]] + [f'"{header}"' for header in headers[1:]])
        f.write(header_line + "\n")
        
        # Process each data row.
        for row in data_rows:
            first_cell = row[0]
            # Check if the first cell is a float that represents an integer.
            if isinstance(first_cell, float) and first_cell.is_integer():
                first_value = str(int(first_cell))
            else:
                first_value = str(first_cell)
                
            # Build the line by converting each of the other cells to strings wrapped in quotes.
            row_line = ";".join([first_value] + [f'"{str(cell)}"' for cell in row[1:]])
            f.write(row_line + "\n")

# Example usage:
# convert_excel_to_csv('input.xlsx', 'output.csv')
convert_excel_to_csv('input.xls', 'output.csv')


In [25]:
# calculate the TIME for conversion 
import python_calamine
import time

def convert_excel_to_csv(input_excel_path: str, output_csv_path: str) -> None:
    """
    Converts the first sheet of an Excel file (XLS/XLSX) to a CSV file with a custom structure.
    
    The output CSV uses a semicolon as a delimiter. The first column is output as an integer (if applicable)
    while every subsequent column is wrapped in double quotes exactly once.
    
    Additionally, the function tracks and prints the time taken to process the data.
    
    Args:
        input_excel_path (str): Path to the input Excel file.
        output_csv_path (str): Path where the output CSV file will be saved.
    """
    start_time = time.time()
    
    with open(input_excel_path, 'rb') as excel_file:
        workbook = python_calamine.CalamineWorkbook.from_filelike(excel_file)
        sheet = workbook.get_sheet_by_index(0)
        data = sheet.to_python()
        
        if not data:
            print("No data found.")
            return
        
        # Convert header row and data rows to strings where appropriate.
        headers = [str(cell) for cell in data[0]]
        data_rows = data[1:]
    
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as f:
        # Build the header: first cell is left as is; subsequent cells wrapped in a single pair of double quotes.
        header_line = ";".join([headers[0]] + [f'"{header}"' for header in headers[1:]])
        f.write(header_line + "\n")
        
        # Process each data row.
        for row in data_rows:
            first_cell = row[0]
            # Check if the first cell is a float that represents an integer.
            if isinstance(first_cell, float) and first_cell.is_integer():
                first_value = str(int(first_cell))
            else:
                first_value = str(first_cell)
                
            # Build the line by converting each of the other cells to strings wrapped in quotes.
            row_line = ";".join([first_value] + [f'"{str(cell)}"' for cell in row[1:]])
            f.write(row_line + "\n")
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Processing completed in {elapsed_time:.2f} seconds.")

# run convertion method
convert_excel_to_csv('input.xlsx', 'output.csv')

Processing completed in 9.63 seconds.


#### this is float

In [22]:
import python_calamine

def convert_excel_to_csv(input_excel_path: str, output_csv_path: str) -> None:
    """
    Converts the first sheet of an Excel file (XLS/XLSX) to a CSV file with a custom structure.
    
    The output CSV uses a semicolon as a delimiter. The first column is left as is, while every
    subsequent column is wrapped in double quotes exactly once.
    
    Args:
        input_excel_path (str): Path to the input Excel file.
        output_csv_path (str): Path where the output CSV file will be saved.
    """
    with open(input_excel_path, 'rb') as excel_file:
        workbook = python_calamine.CalamineWorkbook.from_filelike(excel_file)
        sheet = workbook.get_sheet_by_index(0)
        data = sheet.to_python()
        
        if not data:
            return
        
        # Convert header row and data rows to strings
        headers = [str(cell) for cell in data[0]]
        data_rows = data[1:]
    
    # Open the output file in write mode
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as f:
        # Build the header: first cell as is; subsequent cells wrapped in a single pair of double quotes.
        header_line = ";".join([headers[0]] + [f'"{header}"' for header in headers[1:]])
        f.write(header_line + "\n")
        
        # Process each data row similarly.
        for row in data_rows:
            # Ensure each cell is converted to a string. The first cell remains unquoted.
            row_line = ";".join([str(row[0])] + [f'"{str(cell)}"' for cell in row[1:]])
            f.write(row_line + "\n")

# Example usage:
convert_excel_to_csv('input.xlsx', 'output.csv')


### first row have quatation

In [21]:
import python_calamine

def convert_excel_to_csv(input_excel_path: str, output_csv_path: str) -> None:
    """
    Converts the first sheet of an Excel file (XLS/XLSX) to a CSV file with a custom structure.
    
    The output CSV uses a semicolon as a delimiter. All fields—including the first column—are wrapped in double quotes.
    
    Args:
        input_excel_path (str): Path to the input Excel file.
        output_csv_path (str): Path where the output CSV file will be saved.
    """
    with open(input_excel_path, 'rb') as excel_file:
        workbook = python_calamine.CalamineWorkbook.from_filelike(excel_file)
        sheet = workbook.get_sheet_by_index(0)
        data = sheet.to_python()
        
        if not data:
            return
        
        # Convert header row and data rows to strings.
        headers = [str(cell) for cell in data[0]]
        data_rows = data[1:]
    
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as f:
        # Wrap every header field in quotes.
        header_line = ";".join([f'"{header}"' for header in headers])
        f.write(header_line + "\n")
        
        # Process each row by wrapping every field in quotes.
        for row in data_rows:
            row_line = ";".join([f'"{str(cell)}"' for cell in row])
            f.write(row_line + "\n")

# Example usage:
convert_excel_to_csv('input.xlsx', 'output.csv')


### - load the csv to the specific path

In [27]:
import requests

def upload_file(file_path: str, upload_url: str) -> bool:
    """
    Uploads a file to the specified URL using the HTTP PUT method.

    Args:
        file_path (str): Path to the file to be uploaded.
        upload_url (str): URL to which the file will be uploaded.

    Returns:
        bool: True if the upload was successful, False otherwise.
    """
    try:
        # Open the file in binary read mode
        with open(file_path, 'rb') as file:
            # Set the appropriate headers for the file upload
            headers = {'Content-Type': 'text/csv'}
            # Send the PUT request to upload the file
            response = requests.put(upload_url, data=file, headers=headers)
            # Check if the upload was successful
            response.raise_for_status()
        return True
    except requests.RequestException as e:
        print(f"An error occurred during the file upload: {e}")
        return False


In [28]:
file_path = 'output.csv'
upload_url = 'https://dev21.local.mteam-ssc.com/et_eu/price_import/output_part2.csv'

success = upload_file(file_path, upload_url)
if success:
    print("File uploaded successfully.")
else:
    print("File upload failed.")


An error occurred during the file upload: HTTPSConnectionPool(host='dev21.local.mteam-ssc.com', port=443): Max retries exceeded with url: /et_eu/price_import/output_part2.csv (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1108)')))
File upload failed.
